In [1]:
# Load the advice dataset
with open("filtered.txt", "r") as f: lines = f.readlines()
lines = [line[:-2] for line in lines] # removes the newlines at the end

In [2]:
!pip3 install tensorflow_text>=2.0.0rc0

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

# Load the model
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [4]:
# Encode advice as responses
advice = tf.constant(lines)
advice_embeddings = module.signatures['response_encoder'](
    input=advice,context=advice) # Not sure what context is supposed to be but
                                 # putting the same thing for both seems to work

In [5]:
# Returns a sorted tuple list with advice and
# how "good" they are for the given query
def ask(query):
  # Encode the question (or more aptly an array of the single function)
  question_embeddings = module.signatures['question_encoder'](tf.constant([query]))

  # Get an array of floats for how "good" each adivce is for this query
  # We can do this via dot product because math
  # (and this is secretly a simmilarity problem)
  weights = np.inner(question_embeddings['outputs'],
                     advice_embeddings['outputs'])[0] # [0] because just one query

  # Returns a sorted tuple list with advice and
  # how "good" they are for the given query
  return sorted([(lines[i], weights[i]) for i in range(len(lines))], key=lambda x: -x[1])

In [8]:
# What question do you ask! This is simmilar to what our final user would enter
query = "What should I do if my girlfriend just broke up with me?"
# Gets the top `n` responses
n = 10

# Does the do
print(ask(query)[:n])

[("Take time to mourn what you've lost", 0.3374713), ('Have a good appetite, lots of friends, and keep busy', 0.31954515), ('Make time to cry', 0.31370622), ('Try not to let a week go by without having lunch or coffee with a friend', 0.2796944), ('Keep busy doing what you like', 0.2751931), ('Spend more time doing for those who can do nothing for you', 0.25344852), ("Get to acceptance as fast as possible. By acceptance, I mean welcome everything that happens in your life. The best time to prepare for a crisis is before it comes. An overreaction is an indication you either didn't prepare or lack imagination [to think this couldn't happen]", 0.24635218), ("Have a pet. Life gets lonely sometimes. Pets are reminders of how we're all living things", 0.23694426), ("Focus more on the present - Don't regret the past and don't be anxious about the future. Appreciate what is happening right now", 0.23319468), ('Forgive yourself for your mistakes', 0.2330167)]
